# Install mysql connector

In [1]:
%pip install mysql-connector-python

   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/14.3 MB 16.8 MB/s eta 0:00:01
   --------------------- ------------------ 7.9/14.3 MB 23.2 MB/s eta 0:00:01
   ---------------------------------------  14.2/14.3 MB 26.2 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 24.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Test MySQL Templates

In [1]:
from mysql_template import MySQLTemplate

template = MySQLTemplate()
template.template_select()

{'query': 'SELECT * from current_dept_emp limit 5',
 'result': [(10001,
   'd005',
   datetime.date(1986, 6, 26),
   datetime.date(9999, 1, 1)),
  (10002, 'd007', datetime.date(1996, 8, 3), datetime.date(9999, 1, 1)),
  (10003, 'd004', datetime.date(1995, 12, 3), datetime.date(9999, 1, 1)),
  (10004, 'd004', datetime.date(1986, 12, 1), datetime.date(9999, 1, 1)),
  (10005, 'd003', datetime.date(1989, 9, 12), datetime.date(9999, 1, 1))],
 'message': 'success'}

In [2]:
template.template_limit()

{'query': 'SELECT * from employees limit 5',
 'result': [(10001,
   datetime.date(1953, 9, 2),
   'Georgi',
   'Facello',
   'M',
   datetime.date(1986, 6, 26)),
  (10002,
   datetime.date(1964, 6, 2),
   'Bezalel',
   'Simmel',
   'F',
   datetime.date(1985, 11, 21)),
  (10003,
   datetime.date(1959, 12, 3),
   'Parto',
   'Bamford',
   'M',
   datetime.date(1986, 8, 28)),
  (10004,
   datetime.date(1954, 5, 1),
   'Chirstian',
   'Koblick',
   'M',
   datetime.date(1986, 12, 1)),
  (10005,
   datetime.date(1955, 1, 21),
   'Kyoichi',
   'Maliniak',
   'M',
   datetime.date(1989, 9, 12))],
 'message': 'success'}

In [1]:
from services.mysql_query_generator import MySQLTemplate
import re

template = MySQLTemplate(db_name='world')

def natural_lang_query(input):
    tokens = input.split(" ")
    if "select" in tokens:
        tables = template.get_tables()
        for table in tables:
            if table in tokens:
                params = {
                    "table": table
                }
                result = template.template_select(params=params)
                print(result)
                break
    elif "distinct" in tokens:
        tables = template.get_tables()
        for table in tables:
            if table in tokens:
                columns = template.get_column_names(table)
                for column in columns:
                    if column in tokens:
                        params = {
                            "table": table,
                            "column": column
                        }
                        result = template.template_distinct(params=params)
                        print(result)
                        break
    elif "ranges" in tokens or "range" in tokens or "between" in tokens:
        tables = template.get_tables()
        for table in tables:
            if table in tokens:
                columns = template.get_column_names(table)
                for column in columns:
                    if column in tokens:
                        matches = re.findall(r'\d+', input)
                        start, end = matches
                        params = {
                            "table": table,
                            "column": column,
                            "start": start,
                            "end": end
                        }
                        result = template.template_between(params=params)
                        print(result)
                        break
    elif "where" in tokens or "whose" in tokens:
        tables = template.get_tables()
        for table in tables:
            if table in tokens:
                columns = template.get_column_names(table)
                for column in columns:
                    if column in tokens:
                        value = tokens[-1]
                        params = {
                            "table": table,
                            "column": column,
                            "value": value
                        }
                        result = template.template_where(params=params)
                        print(result)
                        break
    elif "order" in tokens:
        tables = template.get_tables()
        for table in tables:
            if table in tokens:
                columns = template.get_column_names(table)
                for column in columns:
                    if column in tokens:
                        params = {
                            "table": table,
                            "column": column,
                            "order": "ASC"
                        }
                        result = template.template_order_by(params=params)
                        print(result)
                        break


input = "select rows of city"
natural_lang_query(input)

{'query': 'SELECT * FROM `city` LIMIT 5;', 'desc': "Display rows of table 'city'", 'message': 'success'}


In [2]:
input = "distinct District of city"
natural_lang_query(input)

{'query': 'SELECT DISTINCT `District` FROM `city`;', 'desc': "Display distinct values of column 'District' the table 'city'", 'message': 'success'}


In [3]:
input = "display city whose District is Kabol"
natural_lang_query(input)

{'query': "SELECT * FROM `city` WHERE `District` = 'Kabol';", 'desc': "Display those records of table 'city' whose 'District' is 'Kabol'", 'message': 'success'}


In [4]:
input = "display city and order it by District"
natural_lang_query(input)

{'query': 'SELECT * FROM `city` ORDER BY `District` ASC;', 'desc': "Display the records of table 'city',  the records are sorted by 'District'", 'message': 'success'}


In [5]:
input = "display city whose Population ranges from 10000 to 20000"
natural_lang_query(input)

{'query': 'SELECT * FROM `city` WHERE `Population` BETWEEN 10000 AND 20000;', 'desc': "Display rows of table 'city' whose 'Population' ranges from 10000 to 20000", 'message': 'success'}
